inheritance, association, aggregation, composition

VIẾT USE CASE DIAGRAM VÀ CLASS DIAGRAM  CỦA GAME Ô ĂN QUAN VIỆT NAM (Mandarin square capturing) dựa vào code dưới đây: 
```Java
# Game.java
package game.controls;

import game.controls.board.*;
import game.controls.player.Player;

public class Game {
	private Board myBoard;
	private Player player1;
	private Player player2;
	private boolean isP1Turn;
	private boolean waitMove;
	
	public Game() {
		myBoard = new Board();
		player1 = new Player(1);
		player2 = new Player(2);
		isP1Turn = false;
		waitMove = true;
	}
	
	public void restart() {
		myBoard.reset();;
		player1.reset();;
		player2.reset();;
		isP1Turn = false;
		waitMove = true;
	}
	
	public void playGame() {
		if(!waitMove && !myBoard.gameEnd()) {
			if(isP1Turn) {
				player1.makeMove(myBoard);
				if(player1.isTurn() != isP1Turn) {
					waitMove = true;
					isP1Turn = player1.isTurn();
				}
			}
			else {
				player2.makeMove(myBoard);
				if(player2.isTurn() == isP1Turn) {
					waitMove = true;
					isP1Turn = !player2.isTurn();
				}
			}
		}
		if(waitMove && !myBoard.gameEnd() && raiSoi()>0) {
			if(raiSoi() == 1 && isP1Turn) {
				player1.raiSoi(myBoard);
			}
			if(raiSoi() == 2 && !isP1Turn) {
				player2.raiSoi(myBoard);
			}
		}
	}
	
	public boolean waitingForMove() {
		return waitMove;
	}
	
	public boolean isP1Turn() {
		return isP1Turn;
	}
	
	public void setMove(int ci, int d) {
		if(isValidMove(ci)) {
			if(isP1Turn) {
				this.player1.moveSetup(ci, d);
			}
			else {
				this.player2.moveSetup(ci, d);
			}
			this.waitMove = false;
		}
	}
	
	public boolean isValidMove(int oc) {
		boolean ret = false;
		if(this.isP1Turn() && oc<5 && oc>=0 && myBoard.getCells()[oc].getNumberOfStones()>0) {
			ret = true;
		}
		if(!this.isP1Turn() && oc<11 && oc>5 && myBoard.getCells()[oc].getNumberOfStones()>0) {
			ret = true;
		}
		return (ret && this.waitingForMove());
	}
	
	public int raiSoi() {
		int ret = 0;
		boolean p1 = false;
		boolean p2 = false;
		
		for(int i = 0; i<5; i++) {
			if(myBoard.getCells()[i].getNumberOfStones() > 0) {
				p1 = true;
			}
		}
		
		for(int i = 6; i<11; i++) {
			if(myBoard.getCells()[i].getNumberOfStones() > 0) {
				p2 = true;
			}
		}
		
		if(!p1) {
			ret = 1;
		}
		
		if(!p2) {
			ret = 2;
		}
		
		return ret;
	}
	
	public Board getBoard() {
		return this.myBoard;
	}
	
	public Player getPlayer1() {
		return this.player1;
	}
	
	public Player getPlayer2() {
		return this.player2;
	}
	
	public boolean gameEnd() {
		return this.myBoard.gameEnd();
	}
	
	public int getWinner() {
		int ret = -1;
		if(player1.getPoint()>player2.getPoint()) {
			ret = 1;
		}
		else if(player2.getPoint()>player1.getPoint()) {
			ret = 2;
		}
		else {
			ret = 3;
		}
		return ret;
	}
}

```Java
# Player.java

package game.controls.player;

import java.util.*;
import java.util.concurrent.*;
import java.lang.Math;

import game.controls.board.*;

public class Player {
	private ArrayList<Stone> inHand;
	private ArrayList<Stone> taken;
	private int playerId;
	private int dir = -1;
	private int curIndex = -1;
	private int penalty = 0;
	
	public Player(int id) {
		this.inHand = new ArrayList<Stone>();
		this.taken = new ArrayList<Stone>();
		this.playerId = id;
	}
	
	public void reset() {
		this.inHand = new ArrayList<Stone>();
		this.taken = new ArrayList<Stone>();
		curIndex = -1;
		penalty = 0;
	}
	
	public void makeMove(Board b) {
		if(isTurn()) {
			int nextIndex = Math.floorMod(this.curIndex+dir, 12);
			int afterIndex = Math.floorMod(this.curIndex+2*dir, 12);
			BoardCell cur = b.getCells()[curIndex];
			BoardCell next = b.getCells()[nextIndex];
			BoardCell after = b.getCells()[afterIndex];
			int mc = moveCase(b, cur, next, after);
			switch(mc) {
			case 0:
				releaseStone(cur);
				break;
			case 1:
				pickupStones(cur);
				break;
			case 2:
				takeStonesInNext(next, mc==2);
				break;
			case 3:
				takeStonesInNext(next, mc==2);
				break;
			case 4:
				this.curIndex = -1;
				break;
			}
		}
		try {
			TimeUnit.MILLISECONDS.sleep(300);
		}
		catch(Exception e) {
			e.printStackTrace();
		}
	}
	
	public boolean isTurn() {
		return (this.curIndex>=0);
	}
	
	public void raiSoi(Board b) {
		if(playerId == 1) {
			for(int i=0; i<5; i++) {
				b.getCells()[i].getStonesInCell().add(new Stone(false));
			}
		}
		if(playerId == 2) {
			for(int i=6; i<11; i++) {
				b.getCells()[i].getStonesInCell().add(new Stone(false));
			}
		}
		this.penalty ++;
	}
	
	public void pickupStones(BoardCell bc) {
		ArrayList<Stone> cur = bc.getStonesInCell();
		if(bc.isOQuan()) {
			this.curIndex = -1;
		}
		else {
			this.inHand.addAll(cur);
			cur.clear();
			this.curIndex = Math.floorMod((curIndex+dir), 12);
		}
	}
	
	public void releaseStone(BoardCell bc) {
		//System.out.println(this.inHand.size());
		if(this.inHand.size()>0) {
			ArrayList<Stone> cur = bc.getStonesInCell();
			cur.add(this.inHand.get(this.inHand.size()-1));
			this.inHand.remove(this.inHand.size()-1);
			//System.out.println(this.inHand.size());
			this.curIndex = Math.floorMod(this.curIndex+dir, 12);
		}
	}
	
	public void takeStonesInNext(BoardCell next, boolean endTurn) {
		ArrayList<Stone> stones = next.getStonesInCell();
		this.taken.addAll(stones);
		stones.clear();
		if(endTurn) {
			this.curIndex = -1;
		}
		else {
			this.curIndex = Math.floorMod(this.curIndex+2*dir, 12);
		}
	}
	
	public int moveCase(Board b, BoardCell cur, BoardCell next, BoardCell after) { 
		int ret = 0;
		
		if(this.inHand.size() > 0) {
			ret = 0;//Release 1 stone to cell
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones()>0) {
			ret = 1;//Pickup all stones in cell
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() > 0 && after.getNumberOfStones()>0) {
			ret = 2;//Eat all stones in next cell and end turn
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() > 0 && after.getNumberOfStones()==0) {
			ret = 3;//Eat all stones in next cell and continue
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() == 0) {
			ret = 4;//End turn
		}
		
		return ret;
	}
	
	public int getPoint() {
		int ret = 0;
		for(Stone s: this.taken) {
			ret += s.getValue();
		}
		return ret - this.penalty*5;
	}
	
	public int getNumberTakenStones() {
		return this.taken.size();
	}
	
	public void setCurIndex(int curIndex) {
		this.curIndex = curIndex;
	}
	
	public void setDir(int dir) {
		this.dir = dir;
	}
	
	public boolean isValidMove(int ci) {
		boolean ret = true;
		switch(this.playerId) {
		case 1:
			if(ci<0 || ci>4) {
				ret = false;
			}
			
			break;
		case 2:
			if(ci<6 || ci>10) {
				ret = false;
			}
		}
		return ret;
	}
	
	public void moveSetup(int ci, int d) {
		if(isValidMove(ci)) {
			this.curIndex = ci;
			this.dir = d;
		}
	}
	
	public int getPlayerId() {
		return this.playerId;
	}
}

```Java
Board.java
package game.controls.board;

public class Board {
	private BoardCell[] cells;
	
	public Board() {
		cells = new BoardCell[12];
		for(int i = 0; i<12; i++) {
			boolean quan = (i == 5) || (i==11);
			this.cells[i] = new BoardCell(quan);
		}
	}
	
	public void reset() {
		cells = new BoardCell[12];
		for(int i = 0; i<12; i++) {
			boolean quan = (i == 5) || (i==11);
			this.cells[i] = new BoardCell(quan);
		}
	}
	
	public BoardCell[] getCells() {
		return this.cells;
	}
	
	public boolean gameEnd() {
		return (cells[5].getNumberOfStones() == 0 && cells[11].getNumberOfStones() == 0);
	}
}

```Java
# BoardCell.java
package game.controls.board;

import java.util.*;

public class BoardCell {
	private ArrayList<Stone> stonesInCell;
	private boolean isOQuan;
	
	public BoardCell(boolean isOQuan) {
		stonesInCell = new ArrayList<Stone>();
		this.isOQuan = isOQuan;
		if(isOQuan) {
			this.stonesInCell.add(new Stone(true));
		}
		else {
			for(int i=0; i<5; i++) {
				this.stonesInCell.add(new Stone(false));
			}
		}
	}
	
	public ArrayList<Stone> getStonesInCell() {
		return this.stonesInCell;
	}
	
	public int getNumberOfStones() {
		return this.stonesInCell.size();
	}
	
	public int getPoint() {
		int point = 0;
		for(Stone s: this.stonesInCell) {
			point += s.getValue();
		}
		return point;
	}
	
	public boolean isOQuan() {
		return this.isOQuan;
	}
	
}

```Java
# Stone.java
package game.controls.board;

public class Stone {
	private int value;
	private boolean isQuan;
	
	public Stone(boolean isQuan) {
		this.isQuan = isQuan;
		if(isQuan) {
			this.value = 5;
		}
		else {
			this.value = 1;
		}
	}
	
	public int getValue() {
		return this.value;
	}
	
	public boolean isQuan() {
		return this.isQuan;
	}
}



VIẾT USE CASE DIAGRAM VÀ CLASS DIAGRAM  CỦA GAME Ô ĂN QUAN VIỆT NAM (Mandarin square capturing) dựa vào code dưới đây: 
```Java
# Game.java
package game.controls;

import game.controls.board.*;
import game.controls.player.Player;

public class Game {
	private Board myBoard;
	private Player player1;
	private Player player2;
	private boolean isP1Turn;
	private boolean waitMove;
	
	public Game() {
		myBoard = new Board();
		player1 = new Player(1);
		player2 = new Player(2);
		isP1Turn = false;
		waitMove = true;
	}
	
	public void restart() {
		myBoard.reset();;
		player1.reset();;
		player2.reset();;
		isP1Turn = false;
		waitMove = true;
	}
	
	public void playGame() {
		if(!waitMove && !myBoard.gameEnd()) {
			if(isP1Turn) {
				player1.makeMove(myBoard);
				if(player1.isTurn() != isP1Turn) {
					waitMove = true;
					isP1Turn = player1.isTurn();
				}
			}
			else {
				player2.makeMove(myBoard);
				if(player2.isTurn() == isP1Turn) {
					waitMove = true;
					isP1Turn = !player2.isTurn();
				}
			}
		}
		if(waitMove && !myBoard.gameEnd() && raiSoi()>0) {
			if(raiSoi() == 1 && isP1Turn) {
				player1.raiSoi(myBoard);
			}
			if(raiSoi() == 2 && !isP1Turn) {
				player2.raiSoi(myBoard);
			}
		}
	}
	
	public boolean waitingForMove() {
		return waitMove;
	}
	
	public boolean isP1Turn() {
		return isP1Turn;
	}
	
	public void setMove(int ci, int d) {
		if(isValidMove(ci)) {
			if(isP1Turn) {
				this.player1.moveSetup(ci, d);
			}
			else {
				this.player2.moveSetup(ci, d);
			}
			this.waitMove = false;
		}
	}
	
	public boolean isValidMove(int oc) {
		boolean ret = false;
		if(this.isP1Turn() && oc<5 && oc>=0 && myBoard.getCells()[oc].getNumberOfStones()>0) {
			ret = true;
		}
		if(!this.isP1Turn() && oc<11 && oc>5 && myBoard.getCells()[oc].getNumberOfStones()>0) {
			ret = true;
		}
		return (ret && this.waitingForMove());
	}
	
	public int raiSoi() {
		int ret = 0;
		boolean p1 = false;
		boolean p2 = false;
		
		for(int i = 0; i<5; i++) {
			if(myBoard.getCells()[i].getNumberOfStones() > 0) {
				p1 = true;
			}
		}
		
		for(int i = 6; i<11; i++) {
			if(myBoard.getCells()[i].getNumberOfStones() > 0) {
				p2 = true;
			}
		}
		
		if(!p1) {
			ret = 1;
		}
		
		if(!p2) {
			ret = 2;
		}
		
		return ret;
	}
	
	public Board getBoard() {
		return this.myBoard;
	}
	
	public Player getPlayer1() {
		return this.player1;
	}
	
	public Player getPlayer2() {
		return this.player2;
	}
	
	public boolean gameEnd() {
		return this.myBoard.gameEnd();
	}
	
	public int getWinner() {
		int ret = -1;
		if(player1.getPoint()>player2.getPoint()) {
			ret = 1;
		}
		else if(player2.getPoint()>player1.getPoint()) {
			ret = 2;
		}
		else {
			ret = 3;
		}
		return ret;
	}
}

```Java
# Player.java

package game.controls.player;

import java.util.*;
import java.util.concurrent.*;
import java.lang.Math;

import game.controls.board.*;

public class Player {
	private ArrayList<Stone> inHand;
	private ArrayList<Stone> taken;
	private int playerId;
	private int dir = -1;
	private int curIndex = -1;
	private int penalty = 0;
	
	public Player(int id) {
		this.inHand = new ArrayList<Stone>();
		this.taken = new ArrayList<Stone>();
		this.playerId = id;
	}
	
	public void reset() {
		this.inHand = new ArrayList<Stone>();
		this.taken = new ArrayList<Stone>();
		curIndex = -1;
		penalty = 0;
	}
	
	public void makeMove(Board b) {
		if(isTurn()) {
			int nextIndex = Math.floorMod(this.curIndex+dir, 12);
			int afterIndex = Math.floorMod(this.curIndex+2*dir, 12);
			BoardCell cur = b.getCells()[curIndex];
			BoardCell next = b.getCells()[nextIndex];
			BoardCell after = b.getCells()[afterIndex];
			int mc = moveCase(b, cur, next, after);
			switch(mc) {
			case 0:
				releaseStone(cur);
				break;
			case 1:
				pickupStones(cur);
				break;
			case 2:
				takeStonesInNext(next, mc==2);
				break;
			case 3:
				takeStonesInNext(next, mc==2);
				break;
			case 4:
				this.curIndex = -1;
				break;
			}
		}
		try {
			TimeUnit.MILLISECONDS.sleep(300);
		}
		catch(Exception e) {
			e.printStackTrace();
		}
	}
	
	public boolean isTurn() {
		return (this.curIndex>=0);
	}
	
	public void raiSoi(Board b) {
		if(playerId == 1) {
			for(int i=0; i<5; i++) {
				b.getCells()[i].getStonesInCell().add(new Stone(false));
			}
		}
		if(playerId == 2) {
			for(int i=6; i<11; i++) {
				b.getCells()[i].getStonesInCell().add(new Stone(false));
			}
		}
		this.penalty ++;
	}
	
	public void pickupStones(BoardCell bc) {
		ArrayList<Stone> cur = bc.getStonesInCell();
		if(bc.isOQuan()) {
			this.curIndex = -1;
		}
		else {
			this.inHand.addAll(cur);
			cur.clear();
			this.curIndex = Math.floorMod((curIndex+dir), 12);
		}
	}
	
	public void releaseStone(BoardCell bc) {
		//System.out.println(this.inHand.size());
		if(this.inHand.size()>0) {
			ArrayList<Stone> cur = bc.getStonesInCell();
			cur.add(this.inHand.get(this.inHand.size()-1));
			this.inHand.remove(this.inHand.size()-1);
			//System.out.println(this.inHand.size());
			this.curIndex = Math.floorMod(this.curIndex+dir, 12);
		}
	}
	
	public void takeStonesInNext(BoardCell next, boolean endTurn) {
		ArrayList<Stone> stones = next.getStonesInCell();
		this.taken.addAll(stones);
		stones.clear();
		if(endTurn) {
			this.curIndex = -1;
		}
		else {
			this.curIndex = Math.floorMod(this.curIndex+2*dir, 12);
		}
	}
	
	public int moveCase(Board b, BoardCell cur, BoardCell next, BoardCell after) { 
		int ret = 0;
		
		if(this.inHand.size() > 0) {
			ret = 0;//Release 1 stone to cell
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones()>0) {
			ret = 1;//Pickup all stones in cell
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() > 0 && after.getNumberOfStones()>0) {
			ret = 2;//Eat all stones in next cell and end turn
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() > 0 && after.getNumberOfStones()==0) {
			ret = 3;//Eat all stones in next cell and continue
		}
		
		if(this.inHand.size() == 0 && cur.getNumberOfStones() == 0 && next.getNumberOfStones() == 0) {
			ret = 4;//End turn
		}
		
		return ret;
	}
	
	public int getPoint() {
		int ret = 0;
		for(Stone s: this.taken) {
			ret += s.getValue();
		}
		return ret - this.penalty*5;
	}
	
	public int getNumberTakenStones() {
		return this.taken.size();
	}
	
	public void setCurIndex(int curIndex) {
		this.curIndex = curIndex;
	}
	
	public void setDir(int dir) {
		this.dir = dir;
	}
	
	public boolean isValidMove(int ci) {
		boolean ret = true;
		switch(this.playerId) {
		case 1:
			if(ci<0 || ci>4) {
				ret = false;
			}
			
			break;
		case 2:
			if(ci<6 || ci>10) {
				ret = false;
			}
		}
		return ret;
	}
	
	public void moveSetup(int ci, int d) {
		if(isValidMove(ci)) {
			this.curIndex = ci;
			this.dir = d;
		}
	}
	
	public int getPlayerId() {
		return this.playerId;
	}
}

```Java
Board.java
package game.controls.board;

public class Board {
	private BoardCell[] cells;
	
	public Board() {
		cells = new BoardCell[12];
		for(int i = 0; i<12; i++) {
			boolean quan = (i == 5) || (i==11);
			this.cells[i] = new BoardCell(quan);
		}
	}
	
	public void reset() {
		cells = new BoardCell[12];
		for(int i = 0; i<12; i++) {
			boolean quan = (i == 5) || (i==11);
			this.cells[i] = new BoardCell(quan);
		}
	}
	
	public BoardCell[] getCells() {
		return this.cells;
	}
	
	public boolean gameEnd() {
		return (cells[5].getNumberOfStones() == 0 && cells[11].getNumberOfStones() == 0);
	}
}

```Java
# BoardCell.java
package game.controls.board;

import java.util.*;

public class BoardCell {
	private ArrayList<Stone> stonesInCell;
	private boolean isOQuan;
	
	public BoardCell(boolean isOQuan) {
		stonesInCell = new ArrayList<Stone>();
		this.isOQuan = isOQuan;
		if(isOQuan) {
			this.stonesInCell.add(new Stone(true));
		}
		else {
			for(int i=0; i<5; i++) {
				this.stonesInCell.add(new Stone(false));
			}
		}
	}
	
	public ArrayList<Stone> getStonesInCell() {
		return this.stonesInCell;
	}
	
	public int getNumberOfStones() {
		return this.stonesInCell.size();
	}
	
	public int getPoint() {
		int point = 0;
		for(Stone s: this.stonesInCell) {
			point += s.getValue();
		}
		return point;
	}
	
	public boolean isOQuan() {
		return this.isOQuan;
	}
	
}

```Java
# Stone.java
package game.controls.board;

public class Stone {
	private int value;
	private boolean isQuan;
	
	public Stone(boolean isQuan) {
		this.isQuan = isQuan;
		if(isQuan) {
			this.value = 5;
		}
		else {
			this.value = 1;
		}
	}
	
	public int getValue() {
		return this.value;
	}
	
	public boolean isQuan() {
		return this.isQuan;
	}
}


                           +-------------------+
                           |       Game        |
                           +-------------------+
                           | - myBoard: Board  |
                           | - player1: Player |
                           | - player2: Player |
                           | - isP1Turn: bool  |
                           | - waitMove: bool  |
                           +-------------------+
                           | + restart():void       |
                           | + playGame():void      |
                           | + waitingForMove():bool|
                           | + isP1Turn():bool      |
                           | + setMove(ci, d):void |
                           | + isValidMove(oc):bool|
                           | + raiSoi(): int       |
                           | + getBoard():Board     |
                           | + getPlayer1():Player    |
                           | + getPlayer2():Player    |
                           | + gameEnd():bool       |
                           | + getWinner():int    |
                           +-------------------+

                                 ^
                                 |
                                 |
                +-------------------------------------+
                |                Board                |
                +-------------------------------------+
                | - cells: BoardCell[]                |
                +-------------------------------------+
                | + reset():void                            |
                | + getCells(): BoardCell[]            |
                | + gameEnd(): bool                    |
                +-------------------------------------+

                                 ^
                                 |
                                 |
         +-----------------------------------+
         |            BoardCell             |
         +-----------------------------------+
         | - stonesInCell: ArrayList<Stone> |
         | - isOQuan: bool                   |
         +-----------------------------------+
         | + getStonesInCell(): ArrayList<Stone> |
         | + getNumberOfStones(): int              |
         | + getPoint(): int                       |
         | + isOQuan(): bool                        |
         +-----------------------------------+

                                 ^
                                 |
                                 |
         +-----------------------------------+
         |              Stone                |
         +-----------------------------------+
         | - value: int                      |
         | - isQuan: bool                    |
         +-----------------------------------+
         | + getValue(): int                 |
         | + isQuan(): bool                  |
         +-----------------------------------+

                                 ^
                                 |
                                 |
         +-----------------------------------+
         |             Player                |
         +-----------------------------------+
         | - inHand: ArrayList<Stone>        |
         | - taken: ArrayList<Stone>         |
         | - playerId: int                   |
         | - dir: int=-1                     |
         | - curIndex: int=-1                   |
         | - penalty: int=0                    |
         +-----------------------------------+
         | + reset():void                         |
         | + makeMove(b: Board):void             |
         | + isTurn(): bool                  |
         | + raiSoi(b: Board):void                |
         | + pickupStones(bc: BoardCell):void     |
         | + releaseStone(bc: BoardCell):void     |
         | + takeStonesInNext(next: BoardCell, endTurn: bool) |
         | + moveCase(b: Board, cur: BoardCell, next: BoardCell, after: BoardCell): int |
         | + getPoint(): int                 |
         | + getNumberTakenStones(): int     |
         | + setCurIndex(curIndex: int)      |
         | + setDir(dir: int)                |
         | + isValidMove(ci: int): bool      |
         | + moveSetup(ci: int, d: int)      |
         | + getPlayerId(): int              |
         +-----------------------------------+
